# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

import requests

import warnings
warnings.filterwarnings("ignore")

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formataddr

import pandas as pd
import time, requests, csv, json
from datetime import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_colwidth', 10000)

import sys, os, shutil, pdb
sys.path.insert(1, '/Users/muhammadfaisal/Documents/')





from pathlib import Path
import urllib.request

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output
from bs4 import BeautifulSoup


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

try:
  from tabula.io import read_pdf
except:
  %pip install tabula-py
  
try: # running remotely
    from google.colab import drive 
except: # running locally
    from utils import *
    from engine import GDrive
    

In [2]:
TEST_CATALOGS=[]

TEST_CATALOGS=[x.lower() for x in TEST_CATALOGS]

# Local VS Remote

In [3]:
def download_scripts():
    # downloading utils and engine scripts from github
    r=requests.get("https://raw.githubusercontent.com/faysal887/tricyons-analyser-v2/master/engine.py")
    with open('engine.py', 'w') as f: f.write(r.content.decode("utf-8"))

    r=requests.get("https://raw.githubusercontent.com/faysal887/tricyons-analyser-v2/master/utils.py")
    with open('utils.py', 'w') as f: f.write(r.content.decode("utf-8"))

In [4]:
'''
checking whether the code is running on local machine or on collab
'''

LOCAL=None # running code locally or remotely
try:
    drive.mount('/content/drive')

    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = '/content/drive/MyDrive/Colab Notebooks/client_secrets.json'
    gauth = GoogleAuth()
    gauth.GetAuthUrl()
    print('running on collab')
    LOCAL=False
except Exception as e:
    print(e)
    print('running locally')
    LOCAL=True


if LOCAL: # running locally
    try:
        if g: print('GDrive already connected!')
    except: 
        g=GDrive()
        print('created new connection!')
else: # running remotely
    drive.mount('/content/drive')

    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = '/content/drive/MyDrive/Colab Notebooks/client_secrets.json'
    gauth = GoogleAuth()
    print(gauth.GetAuthUrl())
    redirect_url=input('Enter redirect url')
    code=redirect_url.split('code=')[1].split('&scope=')[0]
    gauth.Authenticate(code)
    download_scripts()
    from utils import *
    from engine import GDrive
    g=GDrive(gauth)



name 'drive' is not defined
running locally
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=454591216913-28jl16cqrsojhifrtmopf0ts9mb4281t.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
created new connection!


# Download Catalogs

In [5]:
TMP_DIR='./tmp'
try: shutil.rmtree(TMP_DIR)
except: pass

In [6]:
# main

tmp_dir='./tmp'
merged_dir=f'{tmp_dir}/merged'
online_catalogs=f'{tmp_dir}/online_catalogs'
gdrive_catalogs=f'{tmp_dir}/gdrive_catalogs'

# 1. create /tmp if not
Path(tmp_dir).mkdir(parents=True, exist_ok=True)
Path(merged_dir).mkdir(parents=True, exist_ok=True)
Path(online_catalogs).mkdir(parents=True, exist_ok=True)
Path(gdrive_catalogs).mkdir(parents=True, exist_ok=True)
print('done 1')

# 2. download excel sheet for links and emails 
tree=['Supplier Catalogues', 'AUTOMATION', 'Brands For Auto-Email']
folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
g.download_files(folder_id, tmp_dir)

linksdf=pd.read_excel(f'{tmp_dir}/Catalogs Link.xlsx')
# data=linksdf.dropna(subset=['Link'], axis=0)
data=strip_column_names(linksdf)
print(data.shape)
print('done 2')



done 1
(42, 16)
done 2


In [7]:
# 3. download excel catalogs
df=data[(data.Active==True) & (data.Type=='LINK')]
print(df.shape)
error_catalogs, catalogs=download_online_excel_catalogs(df, online_catalogs, test_catalogs=TEST_CATALOGS)
print('done 3')

catalogs.keys()

(24, 16)
ERROR:  drachmatrading 'URL LINK' <class 'KeyError'> utils.py 344
(82, 4)
ERROR:  ecomwholesaledeals 'DataFrame' object has no attribute 'append' <class 'AttributeError'> utils.py 236
41/24
 ************************************************* 

Error Catalogs:  {'drachmatrading': KeyError('URL LINK'), 'ecomwholesaledeals': AttributeError("'DataFrame' object has no attribute 'append'")}

 ************************************************* 

done 3


dict_keys(['BuyWholesaleToday', 'DallasWholesaleCompany', 'EmpireDistribution', 'EpilsonWholesale', 'GSCommodityTrading', 'KNTradingLLC', 'KoleImports', 'LvDistribution', 'Medcare_Conair', 'Medcare_COTY', 'Medcare_MASTER', 'MinMaxDeals', 'NexDeal_1', 'NexDeal_2', 'PalletFly', 'Shepher_California_Available', 'Shepher_California_Special', 'Shepher_NewJersey_Available', 'Shepher_NewJersey_Special', 'Shepher_NewYork_Special', 'Shepher_NewYork_Website', 'WeinersLtd'])

In [8]:
error_catalogs

{'drachmatrading': KeyError('URL LINK'),
 'ecomwholesaledeals': AttributeError("'DataFrame' object has no attribute 'append'")}

In [9]:
# 4. delete existing supplier folders
tree_base=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL']
df.Distributor.apply(lambda x: g.delete_by_name(tree_base+[x]))
print('done 4')

done 4


In [10]:
# 5. create new supplier folders
tree=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL']
parent_folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
df['folder_id'] = df.Distributor.apply(lambda x: g.create_folder(parent_folder_id, x))
print('done 5')

# 6. create folder for catalog in supplier folder
df['catalog_folder_id'] = df.folder_id.apply(lambda x: g.create_folder(parent_folder_id=x, folder_name='catalog'))
print('done 6')



done 5
done 6


In [11]:
# 7. upload catalog
# _ = df.apply(lambda x: g.upload_file(folder_id=x.catalog_folder_id, file_path=f'{online_catalogs}/{x.Distributor}.xlsx'), axis=1)
for i, row in df.iterrows():
    print(f'{i+1}/{len(df)}', end='\r')
    file_path=f'{online_catalogs}/{row.Distributor}.xlsx'
    if os.path.exists(file_path):
        g.upload_file(folder_id=row.catalog_folder_id, file_path=file_path)
    
print('done 7')


# 8. delete /tmp
# shutil.rmtree(tmp_dir)
# print('done 8')

done 7


In [19]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
                                # PART-2: Start again with data #
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
linksdf=pd.read_excel(f'{tmp_dir}/Catalogs Link.xlsx')
# data=linksdf.dropna(subset=['Link'], axis=0)
data=strip_column_names(linksdf)
if TEST_CATALOGS:
    df=data[(data.Active==True)&(data.Distributor.str.lower().isin(TEST_CATALOGS))].copy()
else:
    df=data[(data.Active==True)].copy()
    
print(df.shape)

(32, 16)


In [20]:
# 8. for all the suppliers in catalogs_all, get their catalog folder_ids 
for i, row in df.iterrows():
    try:
        print(f'{i+1}/{len(df)}', end='\r')
        tree=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL', row.Distributor, 'catalog']
        catalog_folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
        df.loc[i, 'catalog_folder_id']=catalog_folder_id
    except Exception as e:
        print('Error: ', row['Distributor'], e)
print('done 8')

done 8


In [21]:
# 9. loop on supplier folders id df and download all catalogs from gdrive
# _ = df.catalog_folder_id.apply(lambda x: g.download_files(gdrive_folder_id=x, local_folder_path=gdrive_catalogs))

for i, row in df[['Distributor','catalog_folder_id']].iterrows():
    print(f'{i+1}/{len(df)}', end='\r')

    try:
        dist, _id=row['Distributor'], row['catalog_folder_id']
        g.download_files(gdrive_folder_id=_id, local_folder_path=gdrive_catalogs)
        time.sleep(3)
    except Exception as e:
        print('Error: ', dist, e)
print('done 9')

done 9


In [22]:
# 10. create new catalogs dict obj

catalogs={}
for i, fn in enumerate(os.listdir(gdrive_catalogs)):
    print(f'{i+1}/{len(os.listdir(gdrive_catalogs))}', end='\r')

    if '.xlsx' in fn or 'xls' in fn: tmpdf=pd.read_excel(f'{gdrive_catalogs}/{fn}')
    if '.csv'   in fn: tmpdf=pd.read_csv(f'{gdrive_catalogs}/{fn}')

    fn=fn.split('.')[0]

    catalogs[fn]=tmpdf
    # catalogs[fn.replace('.xlsx','').replace('.csv','').replace('.csv','')]=tmpdf

print('done 10')


done 10


In [12]:
# catalogs['MinMaxDeals']=catalogs['MinMaxDeals'].iloc[4:,]
# catalogs['MinMaxDeals']=make_first_row_header(catalogs['MinMaxDeals'])
# catalogs['MinMaxDeals']=strip_column_names(catalogs['MinMaxDeals'])
# catalogs['LvDistribution'].head()

# LvDistribution, MinMaxDeals

In [13]:
# sanity checks to check all the required columns are present
for name, tmpdf in catalogs.items():
    try:
        # if name!='DPCI_Wholesale': continue
        # pdb.set_trace()
        id_columns_name	= df[df.Distributor==name].id_columns_name.item()
        id_columns_type = df[df.Distributor==name].id_columns_type.item()
        price_column    = df[df.Distributor==name].price_column.item()

        if id_columns_name not in tmpdf.columns: print(f'{name} "{id_columns_name}" column missing')
        if price_column not in tmpdf.columns: print(f'{name} "{price_column}" column missing')
    except Exception as e:
        print(f'Supplier: {name}')
        print(f'ERROR: {e}')
        print(f'id_columns_name: {id_columns_name}')
        print(f'id_columns_type: {id_columns_type}')
        print(f'price_column: {price_column}')


        # print(price_column, id_columns_name, id_columns_type)
        # print('\n')


EpilsonWholesale "PRICE PER UNIT" column missing


In [14]:
# 11. label data
df, catalogs=label_data(df, catalogs)
print('done 11')

All Suppliers are Labeled
Error in EpilsonWholesale
'PRICE PER UNIT' <class 'KeyError'> utils.py 164
done 11


In [15]:
# preprocess each supplier separately
for name, tmpdf in catalogs.items():
    if name == 'NUK':                tmpdf=preprocess_nuk(tmpdf)
    if name == 'BGSales':            tmpdf=preprocess_bgsales(tmpdf)
    if name == 'EcomWholesaleDeals': tmpdf=preprocess_ewd(tmpdf)

    catalogs[name]=tmpdf

In [16]:
# label encode supplier name in df dataframe
# keys=df.Distributor.unique().tolist()
# values=['TRI'+str(i).zfill(5) for i in range(len(keys))]
# labels_map=dict(zip(keys, values))
# df['distributor_key'] = df.Distributor.map(labels_map)

# now this is being done manually
# print('done 12')


# apply labels_map for each catalog in catalog dict of dfs
for key, tmpdf in catalogs.items():
    tmpdf['distributor_key']=df[df.Distributor==key].distributor_key.item()
    catalogs[key]=tmpdf


# 12. upload updated catalog_links df
# df.drop(['folder_id', 'catalog_folder_id'], axis=1).to_excel(f'{tmp_dir}/Catalogs Link.xlsx', index=False)

In [17]:
# 13. check for missing values in each column
missing_total=0
for name, tmpdf in catalogs.items():
    try:
        # print(name, tmpdf['each_price'].isnull().sum())
        missing_total+=tmpdf['each_price'].isnull().sum()
    except Exception as e:
        print(f'{name}: {e}')
print(missing_total)
print('done 12')

EpilsonWholesale: 'each_price'
921
done 12


In [18]:
# 13. preprocess asins
asins_suppliers=df[df.id_columns_type=='asin'].Distributor.tolist()

tmpli=[]
for x in asins_suppliers:
  try:
    print(x, catalogs[x].shape)
    tmpli.append(catalogs[x][['asin', 'each_price', 'distributor_key']])
  except Exception as e:
    print(f'Error in {x}: ', e)

if tmpli:    
  asinsdf = pd.concat(tmpli)
  print('after concating all asinsdf: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

  asinsdf=asinsdf.dropna()
  print('after dropna: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

  asinsdf['each_price']=asinsdf.each_price.apply(pd.to_numeric, errors="coerce")
  asinsdf=asinsdf.dropna(subset=['each_price'])
  print('after price column check: ',asinsdf.shape)

  asinsdf=preprocess_price(asinsdf)
  print('after preprocess_price: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

  asinsdf=asinsdf.reset_index(drop=True)
  print('after reset_index: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

  asinsdf.to_excel(f'{merged_dir}/asinsdf.xlsx', index=False)
else:
  asinsdf=pd.DataFrame()

print('after saving: ', asinsdf.shape)
print('done 13')

BuyWholesaleToday (126, 8)
DallasWholesaleCompany (1013, 10)
Error in DrachmaTrading:  'DrachmaTrading'
EmpireDistribution (31, 10)
EpilsonWholesale (17, 9)
Error in EpilsonWholesale:  "['each_price'] not in index"
GSCommodityTrading (178, 12)
KNTradingLLC (979, 10)
LvDistribution (120, 14)
MinMaxDeals (243, 10)
NexDeal_1 (230, 16)
NexDeal_2 (208, 16)
PalletFly (9333, 28)
after concating all asinsdf:  (12461, 3) 919
after dropna:  (11542, 3) 0
after price column check:  (9831, 3)
after preprocess_price:  (9831, 3) 0
after reset_index:  (9831, 3) 0
after saving:  (9831, 3)
done 13


In [19]:
# 14. preprocess upcs
upc_suppliers=df[df.id_columns_type=='upc'].Distributor.tolist()
# upcsdf = pd.concat([catalogs[x][['upc', 'each_price', 'distributor_key']] for x in upc_suppliers])
tmpli=[]
for x in upc_suppliers:
  try:
    tmpli.append(catalogs[x][['upc', 'each_price', 'distributor_key']])
  except Exception as e:
    print(f'Error in {x}: {e}')

if tmpli:
  upcsdf = pd.concat(tmpli)
  print('after concat: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf=upcsdf.dropna(subset=['upc', 'each_price'])
  print('after dropna: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf=preprocess_upc(upcsdf)
  print('after preprocess_upc: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf=preprocess_price(upcsdf)
  print('after preprocess_price: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf=upcsdf.reset_index(drop=True)
  print('after reset_index: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf.to_excel(f'{merged_dir}/upcsdf.xlsx', index=False)
else: 
  upcsdf=pd.DataFrame()
  
print('after save: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

print('done 14')

Error in EcomWholesaleDeals: 'EcomWholesaleDeals'
after concat:  (91721, 3) 2
after dropna:  (91719, 3) 0
after preprocess_upc:  (90409, 3) 0
after preprocess_price:  (90409, 3) 0
after reset_index:  (90409, 3) 0
after save:  (90409, 3) 0
done 14


In [20]:
# 15. upload merged asin and upc on gdrive
tree=['Supplier Catalogues', 'AUTOMATION', 'ScanUnlimited_ALL']
folder_id=g.find_gdrive_folder_id(tree, folder_id='root')

# delete existing ScanUnlimited_ALL
g.delete_by_id(folder_id)

# create new ScanUnlimited_ALL
tree=['Supplier Catalogues', 'AUTOMATION']
parent_folder_id=g.find_gdrive_folder_id(tree, folder_id='root')
new_folder_id=g.create_folder(parent_folder_id, 'ScanUnlimited_ALL')

if not asinsdf.empty: g.upload_file(new_folder_id, f'{merged_dir}/asinsdf.xlsx')
if not upcsdf.empty: g.upload_file(new_folder_id, f'{merged_dir}/upcsdf.xlsx')
print('done 16')

done 16


In [21]:
print(f'''
    Total:   {asinsdf.shape[0]+upcsdf.shape[0]}
    upcsdf:  {upcsdf.shape[0]}
    asinsdf: {asinsdf.shape[0]}
''')


    Total:   100240
    upcsdf:  90409
    asinsdf: 9831



In [22]:
try: shutil.rmtree(TMP_DIR)
except: pass

# TODO

In [ ]:
# add sheet name column so it shortens the download function
# change ignore to active
# add other columns list column

# Send Emails

In [32]:
def send_email(addr, subject, body):
    try:
        fromaddr = "contact@tricyons.com"
        pswd     = "tricyons123"

        msg = MIMEMultipart()
        msg['From'] = formataddr(('Tricyons LLC', fromaddr))
        msg['To'] = addr
        msg['Subject'] = subject
        body = body

        msg.attach(MIMEText(body, 'plain'))
        # smtpserver = smtplib.SMTP("mail.tricyons.com", 465) # tls

        server = smtplib.SMTP("mail.tricyons.com", 26) # non-tls
        server.starttls()
        server.login(fromaddr, pswd)

        text = msg.as_string()
        server.sendmail(fromaddr, addr, text)
        server.quit()    
        return True
    except Exception as e:
        print('*****************************************')
        print(addr)
        print(e)
        print('*****************************************\n\n')
        return False


### Ask Catalog

In [ ]:
emailsdf=data[(data.Active==True) & (data.Type=='EMAIL')].sort_values('Email', ascending=False)
print(emailsdf.shape)
emailsdf

In [17]:
with open('./email_catalog.txt', 'r') as f:
    email_body_template = f.read()

print(email_body_template)

In [ ]:
email_subject='Updated Catalog'

for i, row in emailsdf.iterrows():
    print(f'{i+1}/{emailsdf.shape[0]}', end='\r')

    email_to=row.Email.strip().lower()

    email_body=email_body_template.replace('SUBJECT_NAME_TAG', row.Email_Subject)

    email_sent = send_email(email_to, email_subject, email_body)
    if not email_sent: print(f'Error in sending email to: {row.Email_Subject}')

    time.sleep(3)

### New Suppliers

In [37]:
data=pd.read_excel('/Users/muhammadfaisal/Downloads/SmartScout - Suppliers.xlsx')
print(data.shape)

data.head(2)

(94, 10)


,Menu,Name,Email,Description,Phone Number,Street,City,State,Country,Zip Code
0,NaN,E-Tech Galaxy,accounts@etechgalaxy.com,E-Tech Galaxy is an experienced consumer electronics and computer industry distributor and wholesaler. Our humble beginnings and commitment to superior service since 2002 led us to expand into major online distribution channels.,817-551-7200,NaN,Forth Worth,TX,US,76140
1,NaN,USA Cosmetics,Admin@usacosmetics.com,"Offers distribution of the largest selection of Korean cosmetics for the specialty retail market in the United States. Products include G9Skin, Benton, CosRx and more. Wholesale partners accepted.",877-488-7275,"5000 Eldorado Pkwy, Ste 150",Frisco,TX,US,75033


In [38]:
df=data[~data.Email.isnull()]
df.shape

(36, 10)

In [39]:
with open('./email_template_new_account.txt', 'r') as f:
    email_body_template = f.read()

print(email_body_template)

Dear SUPPLIER_NAME_TAG,

We are looking to open a Wholesale Account with you.

I am Usman, a representative of Tricyons LLC based in Dallas, Austin. We mainly sell products through our website but 3rd party as well, like Ebay and Amazon.

Can you please provide us your catalog in Excel format with wholesale prices.

Additionally, we are offering our resources for marketing and awareness of the brand.

I’d gladly respond to any question you may have. 

Tax ID # 3-20827-2711-9

Thank you.

Muhammad Usman
Tricyons LLC
Email: https://tricyons.com/
Phone: +1 972 5918591
Address: 11816 Inwood Rd, Ste 5203, Dallas, TX 75244


In [40]:
email_subject='Wholesale Account Opening'

for i, row in df.iterrows():
    print(f'{i+1}/{df.shape[0]}', end='\r')

    email_to=row.Email.strip().lower()

    email_body=email_body_template.replace('SUPPLIER_NAME_TAG', row.Name)

    email_sent = send_email(email_to, email_subject, email_body)

    if not email_sent: print(f'Error in sending email to: {email_to}')

    time.sleep(3)

### Account Open

In [ ]:


with open('./email_template.txt', 'r') as f:
    email_body_template = f.read()

print(email_body_template)


# email_to='faysalaslam887@gmail.com'
# email_subject = "Testing Email Body 3 Spaces"
# email_body=email_body_template.replace('COMPANY_NAME_TAG', 'Test Company')
# send_email(email_to, email_subject, email_body)

# g=GDrive()


'''
tmp_dir='./tmp/'

tree=['Supplier Catalogues', 'Brands For Auto-Email']
g.supplier_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
g.download_files(g.supplier_id, tmp_dir)

all_suppliers=pd.read_excel(f'{tmp_dir}/All Suppliers.xlsx')
print(all_suppliers.shape)

all_suppliers.head()

df=all_suppliers.dropna(subset=['Email'], axis=0).reset_index(drop=True)
print(df.shape)
df.head()

# email_subject = "Wholesale Account Opening"

# for i, row in df.iloc[78:].iterrows():
#     print(f'{i+1}/{df.shape[0]}', end='\r')

#     email_to=row.Email.strip().lower()

#     supplier_name = row.Name.strip()

#     email_body=email_body_template.replace('COMPANY_NAME_TAG', supplier_name)

#     email_sent = send_email(email_to, email_subject, email_body)
#     if not email_sent: print(f'Error in sending email to: {supplier_name}')

#     time.sleep(3)
    
'''

# Test New Catalogs

'Worksheet Wholesale Best-Selling List does not exist.'
(0, 0)
(98545, 25)


In [47]:
from utils import *

In [80]:
url=close

In [69]:
import requests, json

import urllib.request
response = urllib.request.urlopen(best)
html = response.read()

In [71]:
import urllib.request
urllib.request.urlretrieve(url, 'test.xlsx')

('test.xlsx', <http.client.HTTPMessage at 0x7f77ea643ee0>)

In [81]:
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)
df1.head()


(12, 2)


,,"PLEASE DO NOT HIT BUTTON ""SHARE"" AND ASK FOR EDIT ACCESS. To Edit the file, please go to File - Make a copy. And the file is yours to edit"
0,,
1,IMPORTANT READ:,
2,,We are not responsible for products that could...
3,,If you need us to provide shipping quote from ...
4,,


In [83]:
df2=get_catalog_google_sheets_2(url, sheet_name='Has UPC')
print(df2.shape)
df2.head()

(961, 26)


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,None,None,None,None,None,None,Price is Per Unit. And shipping Cost is extra,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,ProductId,Manufacture Name,Typical Lead Time,UPC,Title,Case size,Wholesale Price (over $500 purchase). First or...,Img,Pic,None,...,None,None,None,None,None,None,None,None,None,None
2,MCMAC45973NA01,ON-FILE,1-3 days,NA,Mack Truck Door Check Retainer 23758116,1.0,2.0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,MCBD 22122NA01,ON-FILE,1-3 days,632615422122,BD IMS RELAXATION MASK LAVENDER,1.0,2.0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,MCSUG05814NA01,ON-FILE,1-3 days,854764005814,"Sugar Cookie- Bela Premium 0.5 fl. Oz., Fragra...",1.0,1.04,https://i.ebayimg.com/images/g/DSMAAOSw9jNgN9V...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [66]:
df3=get_catalog_google_sheets_3(url)
df3.shape


(0, 0)

In [67]:
df4=get_catalog_google_sheets_4(url)
print(df4.shape)
df4.head()

'utf-8' codec can't decode byte 0x9d in position 54: invalid start byte
(0, 0)


""


In [6]:
# EpilsonWholesale
df2=get_catalog_google_sheets_2(best, sheet_name='Wholesale Best-Selling List')
# df2=df2.iloc[1:]
# df2=make_first_row_header(df2)
# df2=df2[~df2.ASIN.astype(str).str.contains('--')]
# df2=df2.dropna()

print(df2.shape)
df2.head()

Excel file format cannot be determined, you must specify an engine manually.
(0, 0)


""


In [ ]:
# GSCommodityTrading
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)
df1=df1.iloc[9:]
df1.head(20)


In [ ]:
# MinMaxDeals
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)

In [ ]:
# KNTradingLLC
df2=get_catalog_google_sheets_2(url, sheet_name='Sheet1')
df2=make_first_row_header(df2)
df2=df2[[x for x in df2.columns.tolist() if x!=None]]


In [ ]:
# EcomWholesaleDeals 



In [ ]:
links={
    'EpilsonWholesale': "https://img1.wsimg.com/blobby/go/271277f5-5f94-472c-b153-578386146f02/downloads/COSMETIC%20SPREED%20SHEET%20%20(1).xlsx?ver=1680857754932",
    'GSCommodityTrading': "https://docs.google.com/spreadsheets/d/1EJ8kbB89SYv5fYeeCgds5ue7p_3nyckyN_yujUquJeo/edit#gid=0",
    'MinMaxDeals': "https://docs.google.com/spreadsheets/d/151_G7FTLfJP7y9kypBEF4sYN7gICIrYT/edit",
    'KNTradingLLC': "https://06315f73-014b-478e-b312-e53df7038d3a.filesusr.com/ugd/cc4fa6_9d8a073014d4499d8d117b6724916ea8.xlsx?dn=3_23%20KN%20INVENTORY.xlsx",
    'EcomWholesaleDeals': '1EIzkO1ES0v1VcabxOUQ7VaREIjMnxSgB'
}


In [ ]:
url=links['GSCommodityTrading']
url

In [ ]:
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)
df1=df1.iloc[9:]
df1.head()


In [ ]:
df2=get_catalog_google_sheets_2(url, sheet_name='Sheet1')
df2=make_first_row_header(df2)

df2=df2[[x for x in df2.columns.tolist() if x!=None]]
# df2=df2.dropna()

print(df2.shape)
df2.head()

In [ ]:
df3=get_catalog_google_sheets_3(url)
df3.shape


In [ ]:
df4=get_catalog_google_sheets_4(url)
print(df4.shape)
df4.head()

In [101]:
[ x.lower() for x in ['Medcare_BestSelling', 'Medcare_MASTER', 'Medcare_CLOSEOUT']]

['medcare_bestselling', 'medcare_master', 'medcare_closeout']

In [94]:
best = "https://docs.google.com/spreadsheets/d/1dB6cI-7zBOBjll1XTV1Yav1nP_3xPEPdfK0TTZo_yHA/export?format=xlsx&gid=1640087997"

best=get_catalog_google_sheets_2(best, sheet_name='Wholesale Best-Selling List')

best=best.iloc[1:,]

best=make_first_row_header(best)
best=strip_column_names(best)

print(best.shape)
best.head()

(1127, 22)


1,ProductId,Manufacture Name,Typical Lead Time,UPC,Title,Case size,Wholesale Price (over $500 purchase). First order only required min $350 & at least 3 different items,"Amazon Reference - This is for your convenience ONLY. Please do your own research as Amazon listing could change anytime: asin, upc, pack size, price, picture, color, version, etc...",Unnamed,Unnamed,...,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed
2,15762.0,ON-FILE,2-4 Weeks,035046002626,VIT E OINT W/PABA & ALOE,1.0,5.03,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,16393.0,ON-FILE,2-4 Weeks,740985223611.0,SLIMMING TEA HONEY LEMON,1.0,3.46,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,16624.0,ON-FILE,2-4 Weeks,837864000293.0,MAGNESIUM OXIDE 400MG,1.0,3.56,https://amazon.com/dp/B001UWVIT4/,None,None,...,None,None,None,None,None,None,None,None,None,None
5,16631.0,ON-FILE,2-4 Weeks,837864000408.0,OYSTER SHELL CAL 500+D CHEW,1.0,3.61,https://www.amazon.com/dp/B001QJ53BA/,None,None,...,None,None,None,None,None,None,None,None,None,None
6,16634.0,ON-FILE,2-4 Weeks,837864000521.0,THERAGRAN-M TABS,1.0,5.58,https://www.amazon.com/dp/B001VFNBNG/,None,None,...,None,None,None,None,None,None,None,None,None,None


In [96]:
master='https://docs.google.com/spreadsheets/d/1CGv5t5euc-OH64MCpiVGpwYC9ObCm5ZrceVgm_88o8I/export?format=xlsx&gid=1450292243'

master=get_catalog_google_sheets_2(master, sheet_name='Wholesale Master List')

master=master.iloc[1:,]

master=make_first_row_header(master)
master=strip_column_names(master)

print(master.shape)
master.head()


(98545, 25)


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,None,None,None,None,None,None,We sell by case size. But Price is Per Unit. A...,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,ProductId,Manufacture Name,Typical Lead Time,UPC,Title,Case size,Wholesale Price (over $500 purchase). First or...,Over $1000 purchase: 2% discount,Over $3000 purchase: 3% discount,Talk to Sale Rep for additional discount for p...,...,None,None,None,None,None,None,None,None,None,None
2,15003.0,ON-FILE,2-4 Weeks,740985212400,MELATONIN 3MG,1.0,3.46,3.39,12.52,None,...,None,None,None,None,None,None,None,None,None,None
3,15004.0,ON-FILE,2-4 Weeks,740985227213,MELATONIN 3MG,1.0,5.49,5.38,5.33,None,...,None,None,None,None,None,None,None,None,None,None
4,15009.0,ON-FILE,2-4 Weeks,305361395179,ORALTYE FREEZE POPS,1.0,6.62,6.49,6.43,None,...,None,None,None,None,None,None,None,None,None,None


In [98]:
close='https://docs.google.com/spreadsheets/d/15493Wi1IdPYtX8oYfsQszOTAUkBkMQYDy31wNI3bsoY/export?format=xlsx&gid=1221906338'

close=get_catalog_google_sheets_2(close, sheet_name='Has UPC')
close=close.iloc[1:,]

close=make_first_row_header(close)
close=strip_column_names(close)

print(close.shape)
close.head()

(961, 26)


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,None,None,None,None,None,None,Price is Per Unit. And shipping Cost is extra,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,ProductId,Manufacture Name,Typical Lead Time,UPC,Title,Case size,Wholesale Price (over $500 purchase). First or...,Img,Pic,None,...,None,None,None,None,None,None,None,None,None,None
2,MCMAC45973NA01,ON-FILE,1-3 days,NA,Mack Truck Door Check Retainer 23758116,1.0,2.0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,MCBD 22122NA01,ON-FILE,1-3 days,632615422122,BD IMS RELAXATION MASK LAVENDER,1.0,2.0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,MCSUG05814NA01,ON-FILE,1-3 days,854764005814,"Sugar Cookie- Bela Premium 0.5 fl. Oz., Fragra...",1.0,1.04,https://i.ebayimg.com/images/g/DSMAAOSw9jNgN9V...,None,None,...,None,None,None,None,None,None,None,None,None,None


(959, 26)


1,ProductId,Manufacture Name,Typical Lead Time,UPC,Title,Case size,Wholesale Price (over $500 purchase). First order only required minimum of $350 & at least 3 different items,Img,Pic,Unnamed,...,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed,Unnamed
2,MCMAC45973NA01,ON-FILE,1-3 days,NA,Mack Truck Door Check Retainer 23758116,1.0,2.0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,MCBD 22122NA01,ON-FILE,1-3 days,632615422122,BD IMS RELAXATION MASK LAVENDER,1.0,2.0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,MCSUG05814NA01,ON-FILE,1-3 days,854764005814,"Sugar Cookie- Bela Premium 0.5 fl. Oz., Fragra...",1.0,1.04,https://i.ebayimg.com/images/g/DSMAAOSw9jNgN9V...,None,None,...,None,None,None,None,None,None,None,None,None,None
5,MCSMO58840NA01,ON-FILE,1-3 days,718122258840,Smokeeze Cigarette Filters,1.0,1.0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,MCFRE02575NA01,ON-FILE,1-3 days,047701002575,DenTek Fresh Clean Floss Picks For Extra Tigh...,1.0,1.78,https://m.media-amazon.com/images/I/71P3rT9rAI...,None,None,...,None,None,None,None,None,None,None,None,None,None
